## PnL Explain :  Estimating PnL using sensitivities and Market Data

For more context and definitions around pnl explained, [check out our article on atoti.io](https://www.atoti.io/pnl-explained-with-atoti/).

### A few definitions:

- [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp) refers to any collection of financial assets such as stocks, bonds and cash. 
- [PnL](https://www.investopedia.com/terms/p/plstatement.asp) is a common term used in trading referring to the total "Profit and Loss" made by a portfolio over a certain time period.
- [Maturity date](https://www.investopedia.com/terms/m/maturitydate.asp) refers to the due date on which a borrower must pay back the principal of a debt, i.e. the initial amount of money borrowed.
- [Tenor](https://www.investopedia.com/terms/t/tenor.asp) refers to the length of time remaining in a contract, while maturity refers to the initial length of the agreement upon its inception. The tenor of a financial instrument declines over time, whereas its maturity remains constant.
- [Yield curve](https://www.investopedia.com/terms/y/yieldcurve.asp) is a graphical representation of interest rates per maturity date.
- [Sensitivity](https://www.investopedia.com/terms/s/sensitivity.asp) is the magnitude of a financial instrument's reaction to changes in underlying factors.
- [Greeks](https://www.investopedia.com/terms/g/greeks.asp) describes the different dimensions of risk involved in taking an options position. 
- [Delta](https://www.investopedia.com/terms/d/delta.asp), in particular, is a first-order greek, and represents the ratio that compares the change in the price of an asset to the corresponding change in the price of its derivative. For example, if a stock option has a delta value of 0.75, this means that if the underlying stock increases in price by 1 dollar per share, the option on it will rise by 0.75 dollars per share.

### Introduction

The PnL explain technique seeks to estimate the daily PnL from the change in the underlying risk factors.
In this case the risk factors are determined by the yield curve plotting interest rates for each tenor.

Usually, a portfolio risk manager will monitor the risk factors that impact his portfolio, instead of monitoring all the positions booked in his portfolio. To assess what will be the value of his portfolio, a portfolio risk manager is interested in performing what-if analysis based on a scenario of the risk factor values.

In [ ]:
import atoti as tt

In [ ]:
session = tt.create_session()

## 1. Create stores with the data

The dataset are publicly available on AWS S3:
  - position :  `s3://data.atoti.io/notebooks/pnl-explained/position_sensitivities.csv`
  - position data: `s3://data.atoti.io/notebooks/pnl-explained/position_data.csv`
  - trading desk: `s3://data.atoti.io/notebooks/pnl-explained/trading_desk.csv`
  - market data: `s3://data.atoti.io/notebooks/pnl-explained/market_data.csv`

Step in this exercise:  
  - Load the data into atoti stores
  - Visualize what in the dataset
  - Join the store together

The purpose is to obtain a schema like that:
 
 ![](target_schema.svg)

## 2. Cube and first vizualisations

Steps:
 - Create a cube on the base store.
 - Use the default measures and hierarchies to do a few plots, for instance the sum of sensi per currency and business units.
 - Create 2 new structured multilevel hierarchies:
     - Portfolio Hierarchy with Asset Class, Sub Asset Class, Fund and Portfolio
     - Trading Book Hierarchy with Business Unit, Sub Business Unit, Trading Desk, Book
 - Create a pivot table using these hierarchies
 - Create the theoretical PnL measure:
     - $Yield Change = last value - start of day $
     - $Delta = sensi \times notional$
     - $Theoretical Pnl = (Delta) \times (Yield Change)$
     - Aggregate the Theoretical PnL with a sum, using tt.agg.sum with a scope origin at the granular levels where the Theretical PnL is computed (currency, curve, tenor, book_id, instrument_code
 - Use this measure in a view

## 3. Simulate the last value

We will now simulate different scenario to what would happen to the PnL if the last value changes on the market.

Such a simulation can be created by setup a simulation called "Curve simulation" which adds some value to the "last.VALUE" measure per tenor and currency

In [ ]:
curve_simulation = position_sensitivity_cube.setup_simulation(
    "Curve Simulation",
    levels=[lvl["tenor"], lvl["currency"]],
    add=[m["last.VALUE"]],
    base_scenario="Last Curve",
)

Use this simulation to create 2 scenarios:
  - Yield Curve Parallel Shift: add `-0.001` to all the tenor for a specific currency

![](parallel-curve.png)

 - Yield curve inversion: add `-0.002` to the late tenors of a specific currency
 
![](yield-curve-inversion.png)



## 4. Create a dashboard

We can now create a dashboard grouping several interesting widget un the Application

In [ ]:
session.url